In [ ]:
# !pip install cryptography
# !pip install PyMySQL
# !pip install SQLAlchemy
# !pip install Flask
# !pip install pandas
!pip install Flask-SQLAlchemy

In [4]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.debug = True
app.config['SQLALCHEMY_DATABASE_URI'] = 'mysql+pymysql://{username}:{password}@{host}:{port}/{db}?charset=utf8'.format(username="kokon",
            password="TrOlOlO123))", host="127.0.0.1", port=3306, db="prediction")
db = SQLAlchemy(app)


In [5]:
class Subject(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    results = db.relationship("Results", back_populates="subject")

class Results(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    not_succes_done = db.Column(db.Boolean, default=False)
    subject_id = db.Column(db.Integer, db.ForeignKey('subject.id'), nullable=False)
    # subject = db.relationship('Subject', backref=db.backref('results', lazy=True, cascade="all"))
    subject = db.relationship("Subject", back_populates="results", cascade="all")
    scores = db.relationship("Scores", back_populates="result")


class Scores(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    result_id = db.Column(db.Integer, db.ForeignKey('results.id'), nullable=False)
    # result = db.relationship('Results', backref=db.backref('scores', lazy=True, cascade="all"))
    result = db.relationship("Results", back_populates="scores", cascade="all")
    visiting = db.Column(db.Float, default=0.0)
    scores_before = db.Column(db.Float, default=0.0)
    scores_count = db.Column(db.Float, default=0.0)
    sqrt_var = db.Column(db.Float, default=0.0)
    scores_test = db.Column(db.Float, default=0.0)
    test_number = db.Column(db.Integer, nullable=False)

In [13]:
import tkinter.filedialog as tkFileDialog
def open_file():
    return tkFileDialog.askopenfilename( title='Выберите файл:')
import pandas as pd

data = pd.read_csv(open_file(), delimiter=";", encoding='cp1251', decimal=",") #delimiter=";", 
data = data.dropna()
data

,ФИО,Пол,Баллы,Не сдал(-а),Посещение до Контрольная работа 1,Баллы до Контрольная работа 1,Количество баллов до Контрольная работа 1,Корень дисперсии баллов до Контрольная работа 1,Контрольная работа 1,Посещение до Контрольная работа 2,Баллы до Контрольная работа 2,Количество баллов до Контрольная работа 2,Корень дисперсии баллов до Контрольная работа 2,Контрольная работа 2
0,Абатуров Георгий Сергеевич,1,76.0,0,1.0,0.0,0.0,0.0,15.0,1.0,20.0,1.0,5.527707983925667,38.0
1,Абдимиталипов Мустафа,1,46.0,1,0.86,2.0,1.0,0.8454843286542927,1.0,0.73,23.0,3.0,4.231987975198208,20.0
2,Абдимуминов Абдужалол Абдихаким угли,1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Абдразакова Екатерина Наилевна,0,100.0,0,0.79,24.0,8.0,1.704772712123232,29.0,1.0,22.0,5.0,5.189465825646755,25.0
4,Абдрашитова Диана Дамировна,0,64.0,0,0.93,0.0,0.0,0.0,9.0,0.55,20.0,1.0,5.527707983925667,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,Ярмоц Надежда Андреевна,0,64.0,0,1.0,5.5,4.0,0.7441175562369161,4.0,0.91,24.5,3.0,5.403001994776195,30.0
949,Ярославцев Дмитрий Сергеевич,1,9.0,1,0.79,2.0,1.0,0.4841229182759271,6.0,0.82,1.0,1.0,0.2763853991962833,0.0
950,Яруллин Тимофей Русланович,1,61.0,0,1.0,2.0,2.0,0.33071891388307384,15.0,1.0,19.0,8.0,2.628635049264584,25.0
951,Ятло Николай Олегович,1,18.0,1,0.93,0.0,0.0,0.0,11.0,0.55,7.0,3.0,1.0374916331657278,0.0


In [7]:
with app.app_context() as ctx:
    ctx.push()
    db.create_all()

In [14]:
subject_temp = Subject(name="Математический анализ")
with app.app_context():
    db.session.add_all([subject_temp])
    db.session.commit()

In [15]:
exams = [name  for name in data.columns.to_list() if name.startswith("Контрольная")]
with app.app_context():
    for index, d in data.iterrows():
        result = Results(not_succes_done= d["Не сдал(-а)"]==1.0, subject=subject_temp)
        db.session.add_all([result])
        db.session.commit()
        for ex in exams:
            score = Scores(result=result, visiting=d[f"Посещение до {ex}"], scores_test = d[ex], test_number=exams.index(ex)+1)
            if (f"Баллы до {ex}" in data.columns):
                score.scores_before = d[f"Баллы до {ex}"]
                score.scores_count = d[f"Количество баллов до {ex}"]
                score.sqrt_var = d[f"Корень дисперсии баллов до {ex}"]
            db.session.add_all([score])
            db.session.commit()